In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr 15 07:19:44 2021

"""
import pandas as pd
import numpy as np
from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer
import warnings; warnings.simplefilter('ignore')


df = pd.read_csv("./movies_metadata.csv", low_memory=False)
df.head()

drop_df = ["homepage", "poster_path", "video", "imdb_id","original_title","spoken_languages", "production_countries", "adult"]
df = df.drop(drop_df, axis=1) # drops the columns from drop_df
df = df.drop_duplicates(keep='first') # removes the duplicates from the df
df.dropna(how="all",inplace=True) # if each column is NaN in a row, drops this row
df.dropna(subset=["title"], inplace=True) # dropNan in title
df["id"] =pd.to_numeric(df['id'], errors='coerce', downcast="integer") #change column to numeric
df["popularity"] =pd.to_numeric(df['popularity'], errors='coerce', downcast="float") 
df['release_date'] = pd.to_datetime(df['release_date'])
df['release_year'] = df['release_date'].dt.year # change to date and gets year
df["status"].fillna(df["status"].value_counts().idxmax(), inplace=True)
df["runtime"] = df["runtime"].replace(0, np.nan)
df["runtime"].fillna(df["runtime"].mean(), inplace=True) 
df.dropna(subset=["release_date"],inplace=True)
df.dropna(subset=["original_language"],inplace=True)

# function for cleaning
def json_to_arr(cell, wanted = "name"): 
    cell = literal_eval(cell)
    if cell == [] or (isinstance(cell, float) and cell.isna()):
        return np.nan
    result = []
    counter = 0
    for element in cell:
        if counter < 3:
            result.append(element[wanted])
            counter += 1
        else:
            break
    return result[:3]

df[['production_companies']] = df[['production_companies']].applymap(json_to_arr) # cleans column

# cleans genres
df['genres'] = df['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
s = df.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = df.drop('genres', axis=1).join(s)

# gets links_small df
links_small = pd.read_csv('links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
df['id'] = df['id'].astype('int')
df2 = df[df['id'].isin(links_small)]

# new text columns
df2['tagline'] = df2['tagline'].fillna('')
df2['description'] = df2['overview'] + df2['tagline']
df2['description'] = df2['description'].fillna('')

# join in credits and keywords tables and change type
credits = pd.read_csv('credits.csv')
keywords = pd.read_csv('keywords.csv')
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
df['id'] = df['id'].astype('int')

df = df.merge(credits, on='id')
df = df.merge(keywords, on='id')

df3 = df[df['id'].isin(links_small)]

# cleans with literal_eval
df3['cast'] = df3['cast'].apply(literal_eval)
df3['crew'] = df3['crew'].apply(literal_eval)
df3['keywords'] = df3['keywords'].apply(literal_eval)
df3['cast_size'] = df3['cast'].apply(lambda x: len(x))
df3['crew_size'] = df3['crew'].apply(lambda x: len(x))

# get director function from crew column
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

# cleans columns
df3['director'] = df3['crew'].apply(get_director)

df3['cast'] = df3['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
df3['cast'] = df3['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

df3['keywords'] = df3['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

df3['cast'] = df3['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

df3['director'] = df3['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
df3['director'] = df3['director'].apply(lambda x: [x,x, x])


s = df3.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()
s = s[s > 1]

# stemmer for better text similarity
stemmer = SnowballStemmer('english')

# gets filter_keywords
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

df3['keywords'] = df3['keywords'].apply(filter_keywords)
df3['keywords'] = df3['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
df3['keywords'] = df3['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

# metadata text dump for cosine
df3['text'] = df3['keywords'] + df3['cast'] + df3['director'] + df3['genres']
df3['text'] = df3['text'].apply(lambda x: ' '.join(x))

# clean data columns
df3['belongs_to_collection'] = df3['belongs_to_collection'].fillna("None")
df3['belongs_to_collection'] = (df3['belongs_to_collection'] != "None").astype(int)
df3["budget"] =pd.to_numeric(df3['budget'], errors='coerce', downcast="float") 

# scale money function for bad data
def scale_money(num):
    if num < 100:
        return num * 1000000
    elif num >= 100 and num < 1000:
        return num * 10000
    elif num >= 1000 and num < 10000:
        return num *100
    else:
        return num

df3[['budget', 'revenue']] = df3[['budget', 'revenue']].applymap(scale_money)

# weighted voting function for normalization
vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
m = vote_counts.quantile(0.75)

def weighted_rating(data):
    v = data['vote_count'] + 1 # added +1
    R = data['vote_average']
    return (v / (v + m) * R) + (m / (m + v) * C)

df3['weighted_rating'] = df3.apply(weighted_rating, axis=1)

# cleaning final columns and creating ROI column
drop_df = ["crew"]
df3 = df3.drop(drop_df, axis=1)
df3['ROI'] = df3['revenue']/df3['budget']
df3 = df3.loc[df3['revenue'] * df3['budget'] != 0]
df3['production_companies'].replace('', np.nan, inplace=True)
df3.dropna(subset=['production_companies'], inplace=True)

# export
df3.to_csv('clean_metadata2.csv', index=False, encoding='utf-8')

df3.info()
